Chapter 9 Exercises

In [149]:
import numpy as np
from matplotlib.pyplot import subplots, cm
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table


from sklearn.svm import SVC
from ISLP.svm import plot as plot_svm
from sklearn.metrics import RocCurveDisplay


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from ISLP.models import ModelSpec as MS

roc_curve = RocCurveDisplay.from_estimator

print("Done")

Done


7

In [150]:
Auto = load_data('Auto')
Auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
388,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
389,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
390,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [151]:
Auto = Auto[Auto.columns.drop('name')]
Auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1
...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,82,1
388,44.0,4,97.0,52,2130,24.6,82,2
389,32.0,4,135.0,84,2295,11.6,82,1
390,28.0,4,120.0,79,2625,18.6,82,1


In [152]:
Auto['mpg'].median()

22.75

A

In [153]:
Auto['mpg_over_median'] = (Auto['mpg'] > Auto['mpg'].median()) * 1
Auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,mpg_over_median
0,18.0,8,307.0,130,3504,12.0,70,1,0
1,15.0,8,350.0,165,3693,11.5,70,1,0
2,18.0,8,318.0,150,3436,11.0,70,1,0
3,16.0,8,304.0,150,3433,12.0,70,1,0
4,17.0,8,302.0,140,3449,10.5,70,1,0
...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,82,1,1
388,44.0,4,97.0,52,2130,24.6,82,2,1
389,32.0,4,135.0,84,2295,11.6,82,1,1
390,28.0,4,120.0,79,2625,18.6,82,1,1


In [154]:
Auto = Auto[Auto.columns.drop('mpg')]
Auto

,cylinders,displacement,horsepower,weight,acceleration,year,origin,mpg_over_median
0,8,307.0,130,3504,12.0,70,1,0
1,8,350.0,165,3693,11.5,70,1,0
2,8,318.0,150,3436,11.0,70,1,0
3,8,304.0,150,3433,12.0,70,1,0
4,8,302.0,140,3449,10.5,70,1,0
...,...,...,...,...,...,...,...,...
387,4,140.0,86,2790,15.6,82,1,1
388,4,97.0,52,2130,24.6,82,2,1
389,4,135.0,84,2295,11.6,82,1,1
390,4,120.0,79,2625,18.6,82,1,1


B-C

In [155]:
design = MS(Auto.columns.drop('mpg_over_median')).fit(Auto)
Y = np.array(Auto['mpg_over_median'])
X = design.transform(Auto) 

In [156]:
K = 5
kfold = skm.KFold(K, random_state=0, shuffle=True)


Cs = 10. ** np.arange(-3, 4)
gammas = 10. ** np.arange(-3, 3)
degrees = np.arange(2,6)

rbf_grid = {'clf__C':Cs, 'clf__gamma':gammas, 'clf__kernel':['rbf']}
lin_grid = {'clf__C':Cs, 'clf__kernel':['linear']}
poly_grid = {'clf__C':Cs, 'clf__kernel':['poly'], 'clf__degree': degrees }

pipe = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

grid_search = skm.GridSearchCV(pipe, param_grid=[rbf_grid, lin_grid, poly_grid], cv= kfold)

grid_search.fit(X,Y)

grid_search.best_params_

{'clf__C': 1.0, 'clf__gamma': 1.0, 'clf__kernel': 'rbf'}

The 5-fold cross validation presented the above hyperparameters as the best, a radial basis kernel with a gamma of 1 and a c of 1

8

A-E

In [157]:
OJ = load_data('OJ')
OJ

,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,CH,237,1,1.75,1.99,0.00,0.00,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,CH,239,1,1.75,1.99,0.00,0.30,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,CH,245,1,1.86,2.09,0.17,0.00,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,MM,227,1,1.69,1.69,0.00,0.00,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,CH,228,7,1.69,1.69,0.00,0.00,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,CH,252,7,1.86,2.09,0.10,0.00,0,0,0.587822,2.09,1.76,0.33,Yes,0.000000,0.053763,0.23,0
1066,CH,256,7,1.86,2.18,0.00,0.00,0,0,0.670258,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0
1067,MM,257,7,1.86,2.18,0.00,0.00,0,0,0.736206,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0
1068,CH,261,7,1.86,2.13,0.00,0.24,0,0,0.588965,1.89,1.86,0.03,Yes,0.112676,0.000000,0.27,0


In [158]:
OJ['bought_MM'] = (OJ['Purchase'] == 'MM')* 1
OJ

,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE,bought_MM
0,CH,237,1,1.75,1.99,0.00,0.00,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1,0
1,CH,239,1,1.75,1.99,0.00,0.30,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1,0
2,CH,245,1,1.86,2.09,0.17,0.00,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1,0
3,MM,227,1,1.69,1.69,0.00,0.00,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1,1
4,CH,228,7,1.69,1.69,0.00,0.00,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,CH,252,7,1.86,2.09,0.10,0.00,0,0,0.587822,2.09,1.76,0.33,Yes,0.000000,0.053763,0.23,0,0
1066,CH,256,7,1.86,2.18,0.00,0.00,0,0,0.670258,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0,0
1067,MM,257,7,1.86,2.18,0.00,0.00,0,0,0.736206,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0,1
1068,CH,261,7,1.86,2.13,0.00,0.24,0,0,0.588965,1.89,1.86,0.03,Yes,0.112676,0.000000,0.27,0,0


In [159]:
OJ = OJ[OJ.columns.drop('Purchase')]
OJ

,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE,bought_MM
0,237,1,1.75,1.99,0.00,0.00,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1,0
1,239,1,1.75,1.99,0.00,0.30,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1,0
2,245,1,1.86,2.09,0.17,0.00,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1,0
3,227,1,1.69,1.69,0.00,0.00,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1,1
4,228,7,1.69,1.69,0.00,0.00,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,252,7,1.86,2.09,0.10,0.00,0,0,0.587822,2.09,1.76,0.33,Yes,0.000000,0.053763,0.23,0,0
1066,256,7,1.86,2.18,0.00,0.00,0,0,0.670258,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0,0
1067,257,7,1.86,2.18,0.00,0.00,0,0,0.736206,2.18,1.86,0.32,Yes,0.000000,0.000000,0.32,0,1
1068,261,7,1.86,2.13,0.00,0.24,0,0,0.588965,1.89,1.86,0.03,Yes,0.112676,0.000000,0.27,0,0


In [160]:
OJ = OJ[OJ.columns.drop('Store7')]
OJ

,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,PctDiscMM,PctDiscCH,ListPriceDiff,STORE,bought_MM
0,237,1,1.75,1.99,0.00,0.00,0,0,0.500000,1.99,1.75,0.24,0.000000,0.000000,0.24,1,0
1,239,1,1.75,1.99,0.00,0.30,0,1,0.600000,1.69,1.75,-0.06,0.150754,0.000000,0.24,1,0
2,245,1,1.86,2.09,0.17,0.00,0,0,0.680000,2.09,1.69,0.40,0.000000,0.091398,0.23,1,0
3,227,1,1.69,1.69,0.00,0.00,0,0,0.400000,1.69,1.69,0.00,0.000000,0.000000,0.00,1,1
4,228,7,1.69,1.69,0.00,0.00,0,0,0.956535,1.69,1.69,0.00,0.000000,0.000000,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,252,7,1.86,2.09,0.10,0.00,0,0,0.587822,2.09,1.76,0.33,0.000000,0.053763,0.23,0,0
1066,256,7,1.86,2.18,0.00,0.00,0,0,0.670258,2.18,1.86,0.32,0.000000,0.000000,0.32,0,0
1067,257,7,1.86,2.18,0.00,0.00,0,0,0.736206,2.18,1.86,0.32,0.000000,0.000000,0.32,0,1
1068,261,7,1.86,2.13,0.00,0.24,0,0,0.588965,1.89,1.86,0.03,0.112676,0.000000,0.27,0,0


In [161]:
design = MS(OJ.columns.drop('bought_MM'))
X = design.fit_transform(OJ)
Y = np.array(OJ['bought_MM'])

In [162]:
(X_train, X_test, y_train, y_test) = skm.train_test_split(X, Y, test_size=0.2, random_state=0)

In [166]:
grid_search = skm.GridSearchCV(pipe, param_grid= lin_grid, cv= kfold)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', SVC())]),
             param_grid={'clf__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'clf__kernel': ['linear']})

In [168]:
grid_search.best_params_

{'clf__C': 0.01, 'clf__kernel': 'linear'}

In [172]:
Y_hat_test = grid_search.best_estimator_.predict(X_test)
confusion_table(Y_hat_test, y_test)

Truth,0,1
Predicted,,
0,105,21
1,17,71


F-H

In [174]:
Cs = 10. ** np.arange(-3, 4)
gammas = 10. ** np.arange(-3, 3)
degrees = np.arange(2,6)

rbf_grid = {'clf__C':Cs, 'clf__kernel':['rbf']}
lin_grid = {'clf__C':Cs, 'clf__kernel':['linear']}
poly_grid = {'clf__C':Cs, 'clf__kernel':['poly'], 'clf__degree': degrees }

pipe = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

grid_search = skm.GridSearchCV(pipe, param_grid=[rbf_grid, lin_grid, poly_grid], cv= kfold)

grid_search.fit(X_train,y_train)

grid_search.best_params_

{'clf__C': 0.01, 'clf__kernel': 'linear'}

In [175]:
Y_hat_test = grid_search.best_estimator_.predict(X_test)
confusion_table(Y_hat_test, y_test)

Truth,0,1
Predicted,,
0,105,21
1,17,71


The best approach remains the SVM with a linear kernel and a C of 0.01